<a href="https://colab.research.google.com/github/lmencisoe/ADL/blob/main/Project/Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from pandas_profiling import ProfileReport
import tensorflow as tf
from sklearn.feature_extraction import DictVectorizer
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from xgboost import plot_importance
from keras.layers import Dropout, Flatten, Dense, Input, Rescaling, Lambda, MaxPooling2D, Conv2D, AveragePooling2D
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection  import GridSearchCV
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense, Input
from keras.utils.vis_utils import plot_model

import statistics
from matplotlib import pyplot
import seaborn as sns
import shap
import warnings
import re
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [3]:
new_df3 = pd.read_csv("/content/drive/MyDrive/Proyecto_ADL/data_modelo.csv",sep=";")
base_total_max = pd.read_csv("/content/drive/MyDrive/Proyecto_ADL/equipos.csv",sep=";")

In [4]:
X_base_model = new_df3.drop(columns=['team', 'team2', 'results', 'neutral_location'])
Y_model = new_df3['results']
Y_model

0        3
1        1
2        2
3        1
4        2
        ..
15654    1
15655    3
15656    3
15657    3
15658    1
Name: results, Length: 15659, dtype: int64

In [24]:
X_base_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15659 entries, 0 to 15658
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   team1_rank  15659 non-null  float64
 1   team2_rank  15659 non-null  float64
 2   team1_gk    15659 non-null  float64
 3   team2_gk    15659 non-null  float64
 4   team1_de    15659 non-null  float64
 5   team1_of    15659 non-null  float64
 6   team1_mi    15659 non-null  float64
 7   team2_de    15659 non-null  float64
 8   team2_of    15659 non-null  float64
 9   team2_mi    15659 non-null  float64
 10  official_t  15659 non-null  int64  
 11  team1_af    15659 non-null  int64  
 12  team1_as    15659 non-null  int64  
 13  team1_eu    15659 non-null  int64  
 14  team1_na    15659 non-null  int64  
 15  team1_oc    15659 non-null  int64  
 16  team1_sa    15659 non-null  int64  
 17  team2_af    15659 non-null  int64  
 18  team2_as    15659 non-null  int64  
 19  team2_eu    15659 non-nul

In [ ]:
X_base_model['official_t'] = X_base_model['official_t'].astype(float))
X_base_model['team1_af'] = X_base_model['team1_af'].astype(float))
X_base_model['team1_as'] = X_base_model['team1_as'].astype(float))
X_base_model['team1_eu'] = X_base_model['team1_eu'].astype(float))
X_base_model['team1_na'] = X_base_model['team1_na'].astype(float))
X_base_model['team1_oc'] = X_base_model['team1_oc'].astype(float))
X_base_model['team1_sa'] = X_base_model['team1_sa'].astype(float))
X_base_model['team2_af'] = X_base_model['team2_af'].astype(float))
X_base_model['team2_as'] = X_base_model['team2_as'].astype(float))
X_base_model['team2_eu'] = X_base_model['team2_eu'].astype(float))
X_base_model['team2_na'] = X_base_model['team2_na'].astype(float))
X_base_model['team2_oc'] = X_base_model['team2_oc'].astype(float))
X_base_model['team2_sa'] = X_base_model['team2_sa'].astype(float))
X_base_model.info()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_base_model, Y_model, test_size=0.2, random_state=2022)

In [6]:
y_dict = [{'':str(x)} for x in y_train.tolist()]
y_dict_test = [{'':str(x)} for x in y_test.tolist()]
vec = DictVectorizer(sparse=False, dtype=int)
M_train = vec.fit_transform(y_dict)
M_test = vec.fit_transform(y_dict_test)

In [7]:
def get_model(meta, activation, n_layers, hidden_layer_1, hidden_layer_2 = None, hidden_layer_3 = None, hidden_layer_4 = None):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    dims = [hidden_layer_1, hidden_layer_2, hidden_layer_3, hidden_layer_4]

    model = Sequential(name="Red_CV")
    model.add(Dense(n_features_in_, input_shape=X_shape_[1:]))

    for i in range(n_layers):
        model.add(Dense(dims[i], activation=activation))
    
    model.add(Dense(3, activation='softmax'))
    return model

In [8]:
clf = KerasClassifier(
    get_model,
    loss="categorical_crossentropy",
    n_layers=2,
    hidden_layer_1=30, 
    hidden_layer_2=10,
    hidden_layer_3=None,
    metrics=["accuracy"],
    epochs=10,
    activation="relu"
)

clf

KerasClassifier(
	model=<function get_model at 0x7f1f4d2017a0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=categorical_crossentropy
	metrics=['accuracy']
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	n_layers=2
	hidden_layer_1=30
	hidden_layer_2=10
	hidden_layer_3=None
	activation=relu
	class_weight=None
)

In [9]:
ct = ColumnTransformer([
       ('encoder', OneHotEncoder(handle_unknown='ignore'),
       make_column_selector(dtype_include=np.object)),
       ], remainder='passthrough')
estimators = [
              ('encoder', ct),
              ('clf', clf),
              ]

pipe = Pipeline(estimators)

In [23]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12527 entries, 2786 to 4989
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   team1_rank  12527 non-null  float64
 1   team2_rank  12527 non-null  float64
 2   team1_gk    12527 non-null  float64
 3   team2_gk    12527 non-null  float64
 4   team1_de    12527 non-null  float64
 5   team1_of    12527 non-null  float64
 6   team1_mi    12527 non-null  float64
 7   team2_de    12527 non-null  float64
 8   team2_of    12527 non-null  float64
 9   team2_mi    12527 non-null  float64
 10  official_t  12527 non-null  int64  
 11  team1_af    12527 non-null  int64  
 12  team1_as    12527 non-null  int64  
 13  team1_eu    12527 non-null  int64  
 14  team1_na    12527 non-null  int64  
 15  team1_oc    12527 non-null  int64  
 16  team1_sa    12527 non-null  int64  
 17  team2_af    12527 non-null  int64  
 18  team2_as    12527 non-null  int64  
 19  team2_eu    12527 non-n

In [10]:
pipe.fit(X_train, y_train)

Epoch 1/10
392/392 [==============================] - 2s 2ms/step - loss: 0.9280 - accuracy: 0.5711
Epoch 2/10
392/392 [==============================] - 1s 2ms/step - loss: 0.9051 - accuracy: 0.5849
Epoch 3/10
392/392 [==============================] - 1s 2ms/step - loss: 0.9000 - accuracy: 0.5889
Epoch 4/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8971 - accuracy: 0.5884
Epoch 5/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8962 - accuracy: 0.5863
Epoch 6/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8942 - accuracy: 0.5880
Epoch 7/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8927 - accuracy: 0.5884
Epoch 8/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8920 - accuracy: 0.5858
Epoch 9/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8915 - accuracy: 0.5891
Epoch 10/10
392/392 [==============================] - 1s 2ms/step - loss: 0.8906 - accuracy: 0.5882

Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1f4ccbb350>)])),
                ('clf',
                 KerasClassifier(activation='relu', epochs=10, hidden_layer_1=30, hidden_layer_2=10, hidden_layer_3=None, loss='categorical_crossentropy', metrics=['accuracy'], model=<function get_model at 0x7f1f4d2017a0>, n_layers=2))])

In [12]:
y_pred = pipe.predict(X_train)
y_pred
print(classification_report(y_train, y_pred))

392/392 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           1       0.62      0.84      0.71      6096
           2       0.40      0.02      0.03      2799
           3       0.55      0.63      0.58      3632

    accuracy                           0.59     12527
   macro avg       0.52      0.49      0.44     12527
weighted avg       0.55      0.59      0.52     12527



In [13]:
with tf.device('/device:GPU:0'): 
    params = {
              "clf__hidden_layer_1": [8, 100, 10, 5],
              "clf__hidden_layer_2": [8, 100, 10, 5],
              "clf__hidden_layer_3": [8, 100, 10, 5],
              "clf__n_layers": [1,2,3],
              "clf__activation": ["relu", "linear", "sigmoid"],
              "clf__optimizer": ["adam", "sgd"],
              "clf__optimizer__learning_rate": [0.0001, 0.001, 0.1],
              "clf__epochs": [10, 30, 20],
             }
    grid = RandomizedSearchCV(pipe, params, cv=3, scoring='neg_log_loss', n_iter=10, random_state=2022)

In [14]:
with tf.device('/device:GPU:0'): 
  grid.fit(X_train, y_train)

Epoch 1/30
261/261 [==============================] - 1s 2ms/step - loss: 1.1507 - accuracy: 0.4288
Epoch 2/30
261/261 [==============================] - 0s 2ms/step - loss: 1.1030 - accuracy: 0.4488
Epoch 3/30
261/261 [==============================] - 0s 2ms/step - loss: 1.0688 - accuracy: 0.4734
Epoch 4/30
261/261 [==============================] - 0s 2ms/step - loss: 1.0445 - accuracy: 0.4907
Epoch 5/30
261/261 [==============================] - 0s 2ms/step - loss: 1.0271 - accuracy: 0.5060
Epoch 6/30
261/261 [==============================] - 0s 2ms/step - loss: 1.0143 - accuracy: 0.5150
Epoch 7/30
261/261 [==============================] - 0s 2ms/step - loss: 1.0048 - accuracy: 0.5203
Epoch 8/30
261/261 [==============================] - 0s 2ms/step - loss: 0.9975 - accuracy: 0.5246
Epoch 9/30
261/261 [==============================] - 0s 2ms/step - loss: 0.9918 - accuracy: 0.5277
Epoch 10/30
261/261 [==============================] - 0s 2ms/step - loss: 0.9871 - accuracy: 0.5312

In [15]:
best_model = grid.best_estimator_

grid.best_params_

{'clf__optimizer__learning_rate': 0.1,
 'clf__optimizer': 'sgd',
 'clf__n_layers': 2,
 'clf__hidden_layer_3': 100,
 'clf__hidden_layer_2': 10,
 'clf__hidden_layer_1': 8,
 'clf__epochs': 10,
 'clf__activation': 'relu'}

In [16]:
y_pred = best_model.predict(X_train)
y_pred
print(classification_report(y_train, y_pred))

392/392 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           1       0.59      0.90      0.71      6096
           2       0.00      0.00      0.00      2799
           3       0.59      0.52      0.55      3632

    accuracy                           0.59     12527
   macro avg       0.39      0.47      0.42     12527
weighted avg       0.46      0.59      0.51     12527



In [17]:
y_pred = best_model.predict(X_test)
y_pred
print(classification_report(y_test, y_pred))

98/98 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           1       0.58      0.89      0.70      1498
           2       0.00      0.00      0.00       748
           3       0.56      0.51      0.54       886

    accuracy                           0.57      3132
   macro avg       0.38      0.47      0.41      3132
weighted avg       0.44      0.57      0.49      3132



In [18]:
base_total_max['team']

0         Afghanistan
1             Albania
2             Algeria
3      American Samoa
4             Andorra
            ...      
207           Vietnam
208             Wales
209             Yemen
210            Zambia
211          Zimbabwe
Name: team, Length: 212, dtype: object

In [19]:
def obtain_new(equipo1, equipo2):
    base1 = base_total_max[base_total_max['team'] == equipo1]
    base1 = base1.drop(columns=['date'])
    base2 = base_total_max[base_total_max['team'] == equipo2]
    base2 = base2.drop(columns=['date'])
    base2 = base2.rename(columns={"team": "team2",
                                                  "team1_rank": "team2_rank",
                                                  "team1_gk": "team2_gk",
                                                  "team1_de": "team2_de",
                                                  "team1_of": "team2_of",
                                                  "team1_mi": "team2_mi", 
                                                  "team1_af": "team2_af",
                                                  "team1_as": "team2_as",
                                                  "team1_eu": "team2_eu",
                                                  "team1_na": "team2_na",
                                                  "team1_oc": "team2_oc",
                                                  "team1_sa": "team2_sa"})
    base1 = base1.drop(columns=['team'])
    base2 = base2.drop(columns=['team2'])
    base_total = pd.concat([base1.reset_index(drop=True), base2.reset_index(drop=True)], axis=1)
    base_total['official_t'] = 1
    base_total['dif_rank'] = base_total['team1_rank'] - base_total['team2_rank']
    base_total['dif_gf'] = base_total['team1_gk'] - base_total['team2_gk']
    base_total['dif_de'] = base_total['team1_de'] - base_total['team2_de']
    base_total['dif_of'] = base_total['team1_of'] - base_total['team2_of']
    base_total['dif_mi'] = base_total['team1_mi'] - base_total['team2_mi']
    prediccion = best_model.predict_proba(base_total)
    return prediccion

In [20]:
obtain_new('Ecuador', 'Qatar')

1/1 [==============================] - 0s 17ms/step


array([0.6020424 , 0.22836103, 0.16959654], dtype=float32)

In [22]:
obtain_new('Qatar', 'Ecuador')

1/1 [==============================] - 0s 59ms/step


array([0.4547228 , 0.26911253, 0.27616468], dtype=float32)